## nfl lines machine learning project


source data for historical spreads and game scores
http://www.repole.com/sun4cast/data.html

In [7]:
import os

# define the root directory for the nlf code in $MLNLF_ROOT
codeDir = "".join([os.environ['MLNFL_ROOT'], os.path.sep])
dataRoot = "".join([codeDir, "data", os.path.sep])

os.chdir(codeDir)

print(codeDir)
print(dataRoot)

/Users/amit.bhattacharyya/repos/mlnfl/nfl/
/Users/amit.bhattacharyya/repos/mlnfl/nfl/data/


In [8]:
%matplotlib inline

from __future__ import division
from __future__ import print_function
import os
import numpy as np
import pandas as pd
import madden
import sklearn_pandas
import matplotlib.pyplot as plt
import sklearn

import graphlab as gl

In [9]:
# lookup files
from referencedata import ReferenceData
lookupFiles = { 'teams' : {'file': 'nflTeams.csv' },
                'seasons' : {'file': 'seasons.csv' },
               }

lookupDir = "".join([dataRoot, 'lookup', os.path.sep])

print ("lookupFiles = %s" % lookupFiles)
print ("lookupDir = %s" % lookupDir)

lookupFiles = {'seasons': {'file': 'seasons.csv'}, 'teams': {'file': 'nflTeams.csv'}}
lookupDir = /Users/amit.bhattacharyya/repos/mlnfl/nfl/data/lookup/


In [10]:
# in notebook, leave all the diagnostic outputs since this is not the real thing
# we are using the nb to investigate
reference_data = ReferenceData(lookupDir)
reference_data.teams_df.head()

,city,mascot,league,division,year
team,,,,,
Baltimore Ravens,NaN,NaN,afc,north,NaN
New England Patriots,NaN,NaN,afc,east,NaN
Tennessee Titans,NaN,NaN,afc,south,NaN
Atlanta Falcons,NaN,NaN,nfc,south,NaN
Tampa Bay Buccaneers,NaN,NaN,nfc,south,NaN


In [11]:
### NEW multi-season training
import madden
reload(madden)

# need to be arrays
#trainYears = [2013] #range(2008,2013) 

# train on previous 3 yrs of data
testYear = [2014]
trainYears = range(testYear[0]-3,testYear[0]) 


# training data set - includes one extra year for prev yr record
seasons = np.array(trainYears)
print ("training seasons >> ", seasons)

training seasons >>  [2011 2012 2013]


In [12]:
# read all the games
path_to_lines = dataRoot + "lines/"
dfAllGames = madden.readGamesAll(path_to_lines, seasons)

# remove these 2 useless columns for now
dfAllGames = dfAllGames.drop('finalLine',1)
dfAllGames = dfAllGames.drop('yahooLine',1)

# compile season record for all teams
dfAllTeams = madden.seasonRecord(dfAllGames, reference_data)

# apply season records and compute other fields for all games
dfAllGames = madden.processGames(dfAllGames, dfAllTeams, reference_data)
# remove extra year of data 
dfAllGames = dfAllGames[dfAllGames.season.isin(seasons)]

# define independent variables for logistic regression
features = ['favoredRecord','underdogRecord',  # current year records of both teams
            'prevFavoredRecord','prevUnderdogRecord', # prev year records, helps early in season when only few games played
            'gameWeek',  # week in season, should make a good/bad record later in season more important
            'absLine',  # absolute value of spread since favored team already determined
            'divisionGame', # T/F, usually more competitive rivalry games, i.e. bad teams still win home division games.
            'favoredHomeGame', # T/F, important since output of classifier is "did the favored team win?"
            ]
# run the logistic regression
logreg = madden.runML(dfAllGames,features)

# use different test set 
seasonTest = np.array(testYear) # should be only one year
print ("results for >> ", seasonTest)
dfGamesTest = madden.readGamesAll(path_to_lines, seasonTest)
dfTeamsTest = madden.seasonRecord(dfGamesTest,reference_data)
dfGamesTest = madden.processGames(dfGamesTest, dfTeamsTest, reference_data)
dfGamesTest = dfGamesTest[dfGamesTest.season.isin(seasonTest)]

# apply results of logistic regression to the test set
dfPredict = madden.predictGames(dfGamesTest,logreg,features)

# apply ranking logic and determine scoring outcomes for league
dfAll = madden.rankGames(dfPredict,reference_data,seasonTest[0])

madden.py:109: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  team_df['gamesPlayed'] = range(1, len(team_df.index) + 1)  # index 1 thur 16
madden.py:110: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  team_df['team'] = team
madden.py:111: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  team_df['homeGame'] = se

AttributeError: 'module' object has no attribute 'runML'

In [12]:
print (sum(dfPredict.predictWin == dfPredict.favoredWin) / len (dfPredict))


0.753731343284


In [29]:
# use graphlab
glAllGames = gl.SFrame(dfAllGames)

# create graphlab logistic regression model
#gl_model = gl.logistic_classifier.create(glAllGames, 'favoredWin', features=features)

# create graphlab boosted trees regression model
gl_model = gl.boosted_trees_classifier.create(glAllGames, 'favoredWin', features=features)

# create graphlab SVM model
gl_model = gl.svm_classifier.create(glAllGames, 'favoredWin', features=features)

# general classifier
# remove home win 
#glAllGames = glAllGames.remove_column('homeWin')
#glAllGames = glAllGames.remove_column('Visitor Score')
#glAllGames = glAllGames.remove_column('Home Score')

#print (glAllGames.column_names())
gl_model = gl.classifier.create(glAllGames, 'favoredWin', features=features)


PROGRESS: Boosted trees classifier:
--------------------------------------------------------
Number of examples          : 768 (507 positives, 261 negatives)
Number of feature columns   : 8
Number of unpacked features : 8

PROGRESS: Starting Boosted Trees

PROGRESS: --------------------------------------------------------

PROGRESS:   Iter    Accuracy Elapsed time

PROGRESS:      0   7.852e-01        0.00s

...

PROGRESS:      5   8.633e-01        0.01s

PROGRESS:      6   8.633e-01        0.01s

PROGRESS:      7   8.750e-01        0.01s

PROGRESS:      8   8.867e-01        0.02s

PROGRESS:      9   8.971e-01        0.02s

In [6]:
# graphlab training accuracy on training data
gl_predict = gl_model.predict(glAllGames)
ngames = len(glAllGames)
print (sum([ gl_predict == glAllGames['favoredWin'] ]) / ngames)

NameError: name 'gl_model' is not defined

In [31]:
# training accuracy on test data
glGamesTest = gl.SFrame(dfGamesTest)
gl_predict2 = gl_model.predict(glGamesTest)

ngames = len(glGamesTest)
print (sum([ gl_predict2 == glGamesTest['favoredWin'] ]) / ngames)

gl_model.evaluate(glGamesTest)

0.567164179104


{'accuracy': 0.6119402985074627, 'confusion_matrix': Columns:
 	target_label	str
 	predicted_label	str
 	count	int
 
 Rows: 4
 
 Data:
 +--------------+-----------------+-------+
 | target_label | predicted_label | count |
 +--------------+-----------------+-------+
 |      0       |        0        |   18  |
 |      0       |        1        |   18  |
 |      1       |        0        |   34  |
 |      1       |        1        |   64  |
 +--------------+-----------------+-------+
 [4 rows x 3 columns]}

In [32]:
# predict based on probability
gl_probability = gl_model.predict(glGamesTest,output_type='probability')
scoreDiff = glGamesTest['Home Score'] - glGamesTest['Visitor Score']
gl_predict2 = (( scoreDiff * glGamesTest['Line'] * (gl_probability - 0.5) ) > 0)
print (sum([ gl_predict2 == glGamesTest['favoredWin'] ]) / ngames)

0.686567164179


In [34]:
# use margin for computing probability because probability all 0/1
proba_type = "prob"
if proba_type == "margin":
    predict_margin = gl_model.predict(glGamesTest , output_type='margin')
    # rescale margin to mimic probability
    min_max_scaler = sklearn.preprocessing.MinMaxScaler()
    predict_proba = min_max_scaler.fit_transform(predict_margin)

else:
    predict_proba = gl_model.predict(glGamesTest , output_type='probability')



glPredict2 = glGamesTest
glPredict2['predict_proba'] = predict_proba
glPredict2['predict_proba_abs'] = (glPredict2['predict_proba'] - 0.5).apply(lambda x: abs(x))
dfPredict2 = glPredict2.to_dataframe()

# apply ranking logic and determine scoring outcomes for league
dfAll = madden.rankGames(dfPredict2,reference_data,seasonTest[0])

             score  win by   win
lineScore      770     770  True
probaScore1    768     768  True
probaScore2    727     727  True
probaScore3    715     715  True


In [ ]:
# diagnostics ... weekly outcomes
g = dfAll.groupby('gameWeek')['lineScore','probaScore1','probaScore2','probaScore3'].sum()
print(g)
g.sum()

In [ ]:
# predict one week of current season
iweek = 7
reload(madden)

# use different test set 
testYear = [2014]
seasonTest = np.array(testYear) # should be only one year
print ("results for >> ", seasonTest)

dfGamesTest = madden.readGamesAll(path_to_lines, seasonTest)
dfTeamsTest = madden.seasonRecord(dfGamesTest,reference_data)
dfGamesTest = madden.processGames(dfGamesTest, dfTeamsTest, reference_data)
dfGamesTest = dfGamesTest[dfGamesTest.season.isin(seasonTest)]

# pick only this weeks games for predict
dfTest = dfGamesTest[dfGamesTest.gameWeek == iweek]

# apply results of logistic regression to the test set
dfPredict = madden.predictGames(dfTest,logreg,features)
# apply ranking logic and determine scoring outcomes for league
dfAll = madden.rankGames(dfPredict,reference_data,seasonTest[0])



In [ ]:
# display weekly ranking output

# ranking methods choices
# 0. pick based on spread
# 1. always pick favored team, rank by probability of win
# 2. pick winner based on abs(probability - .5), rank by probability
# 3. pick winner based on abs(probability - .5), rank by abs(probability - .5)

rankMethod = 2

dispCols = ['season','gameWeek','Visitor','visitorRecord','Home Team','homeRecord',
            'Line','prevFavoredRecord','prevUnderdogRecord','predict_proba',
            'lineGuess','probaGuess', 'probaAbsGuess', 'predictTeam']


dfAll['predictTeam'] = np.where((dfAll['predict_proba'] - .5) > 0 , dfAll['favorite'], dfAll['underdog'])
guessCol = 'probaGuess'
predictCols = ['predictTeam', 'predict_proba', guessCol, 'favorite','lineGuess', 'Line']
dfAll[predictCols].sort(guessCol, ascending=False)

In [ ]:
# display weekly ranking output for spread method

# ranking methods choices
# 0. pick based on spread
# 1. always pick favored team, rank by probability of win
# 2. pick winner based on abs(probability - .5), rank by probability
# 3. pick winner based on abs(probability - .5), rank by abs(probability - .5)

predictCols = ['favorite','lineGuess', 'absLine','Line', 'favoredHomeGame', 'divisionGame', 'favoredRecord']

sortCols = ['absLine','favoredHomeGame', 'divisionGame', 'favoredRecord', 'favorite']
dfSpread = dfAll[predictCols].sort(sortCols , ascending=False)
#print(dfSpread.to_csv(sys.stdout,sep=',', index=False))
dfSpread

In [ ]:
dfAll.columns

In [ ]:
# display details for a single week
dispWeek = 7
dispCols = ['season','gameWeek','Visitor','Home Team',
            'visitorRecord', 'homeRecord', 'favoredRecord','underdogRecord',
            'prevFavoredRecord','Line','predict_proba','Visitor Score',
            'Home Score','favoredWin','predictWin','lineGuess','probaGuess', 'lineScore','probaScore1','probaScore2']

print(dfAll[dfAll.gameWeek == dispWeek]['lineScore'].sum())
dfAll[dfAll.gameWeek == dispWeek][dispCols]

In [ ]:
# diagnostics ... plot outcomes of games
nWin = dfAllGames.favoredWin.index.tolist()
nLose = dfAllGames[dfAllGames['favoredWin'] == 0].index.tolist()

plt.plot(dfAllGames.favoredRecord[nWin],dfAllGames.underdogRecord[nWin],'bx')
plt.plot(dfAllGames.favoredRecord[nLose],dfAllGames.underdogRecord[nLose],'rx')

In [ ]:
# diagnostics ... logreg variables

# print classifiers
print("classifiers > ",logreg.classes_)

# coefficients
cc = logreg.coef_
cc.shape
print("coeffs")
for ii,ff in enumerate(features):
    print(ii,ff,cc[0][ii])
    

In [ ]:
# run one year of rolling training, control width of training window w/ trainFreq
import runMadden
reload(runMadden)
reload(madden)

# need to be arrays
trainYears = [2012] #range(2008.2013) 
testYear = [2013]
trainFreq = 6

dd = runMadden.runSeasonRolling(trainYears,testYear,olookups,trainFreq)
dd

In [ ]:
# loop over years to test trainFreq

reload(runMadden)
reload(madden)

ddd = None
for y in range(2008,2013):
    trainYears = [y]
    testYear = [y+1]
    trainFreq = 20
    dd = runMadden.runSeasonRolling(trainYears,testYear,olookups,trainFreq)
    if ddd is None:
        ddd = dd
    else:
        ddd = ddd.append(dd)
        
ddd        

In [ ]:
# diagnostics ... plot outcomes of games
nWin = dfAll.favoredWin.index.tolist()
nLose = dfAll[dfAll['favoredWin'] == 0].index.tolist()

plt.plot(dfAll.favoredRecord[nWin],dfAll.underdogRecord[nWin],'bx')
plt.plot(dfAll.favoredRecord[nLose],dfAll.underdogRecord[nLose],'rx')

# pct correct according to spread
print (len(nWin)/ (len(nWin) + len(nLose)))

# pct correct according to model
nWin = dfAll.predictWin.index.tolist()
nLose = dfAll[dfAll['predictWin'] == 0].index.tolist()
print (len(nWin)/ (len(nWin) + len(nLose)))
